In [1]:
from pymodulon.io import *
from pymodulon.motif import *
from os import path

# Load Data

In [2]:
ica_data = load_json_model(os.path.join('../..','data','processed','modulome','salb.json.gz'))

In [3]:
fasta_file = "../../data/external/modulome/genome.fasta"

# Search for motifs

In [4]:
# read gene_nfo5.csv
gene_info = pd.read_csv("../../data/processed/modulome/gene_info6.csv",index_col=0)

In [5]:
gene_info.head()  

,gene_name,eggNOG_OGs,Description,GOs,EC,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,...,BiGG_Reaction,PFAMs,accession,old_locus_tag,start,end,strand,gene_product,COG,operon
gene_id,,,,,,,,,,,,,,,,,,,,,
XNR_RS30570,XNR_RS30570,"COG1672@1|root,COG1672@2|Bacteria",-,-,-,-,-,-,-,-,...,-,"AAA_22,TIR_2,TniB",NC_020990.1,XNR_0001,536,1609,-,hypothetical protein,No COG Annotation,Op0
XNR_RS00010,XNR_RS00010,"COG3935@1|root,COG3935@2|Bacteria,2IDRB@201174...",DnaD domain protein,-,-,-,-,-,-,-,...,-,-,NC_020990.1,NaN,3324,3533,-,hypothetical protein,"Replication, recombination and repair",Op2
XNR_RS00015,XNR_RS00015,"28PSB@1|root,2ZCDV@2|Bacteria,2GNDZ@201174|Act...",DDE superfamily endonuclease,-,-,-,-,-,-,-,...,-,"DDE_Tnp_1,DDE_Tnp_4,HTH_Tnp_4",NC_020990.1,XNR_0003,3671,4441,-,IS5-like element IS112 family transposase,Function unknown,Op3
XNR_RS00020,XNR_RS00020,"2AKX1@1|root,31BQI@2|Bacteria,2GNMY@201174|Act...",-,-,-,-,-,-,-,-,...,-,-,NC_020990.1,XNR_0004,4485,5069,-,hypothetical protein,No COG Annotation,Op4
XNR_RS00025,XNR_RS00025,"COG2944@1|root,COG2944@2|Bacteria,2GM0F@201174...",sequence-specific DNA binding,-,-,-,-,-,-,-,...,-,"DnaB_C,HTH_3,HTH_31",NC_020990.1,XNR_0005,5066,5983,-,dihydrolipoamide S-succinyltransferase,Transcription,Op4


In [6]:
ica_data.gene_table['gene_name'] = ica_data.gene_table['gene_name'].fillna('unknown')

In [ ]:
for k in ica_data.imodulon_names:
    try:
        find_motifs(ica_data, k, fasta_file, force=True)
    except ValueError as e:
        print(f"Skipping {k} due to error: {e}")

In [9]:
ica_data.motif_info

{'BGC-19-20 deletion': <MotifInfo with 6 motifs across 159 sites>,
 'Glutamine': <MotifInfo with 1 motif across 20 sites>,
 'Prophages': <MotifInfo with 8 motifs across 53 sites>,
 'Surugamide repressor': <MotifInfo with 3 motifs across 12 sites>,
 'ArsR': <MotifInfo with 2 motifs across 4 sites>,
 'Paulomycin-1': <MotifInfo with 10 motifs across 98 sites>,
 'UC-1': <MotifInfo with 4 motifs across 430 sites>,
 'Cyclofaulknamycin-1': <MotifInfo with 1 motif across 10 sites>,
 'Leucine': <MotifInfo with 6 motifs across 36 sites>,
 'UC-2': <MotifInfo with 4 motifs across 336 sites>,
 'Histidine': <MotifInfo with 4 motifs across 25 sites>,
 'UC-3': <MotifInfo with 5 motifs across 39 sites>,
 'Cysteine': <MotifInfo with 2 motifs across 16 sites>,
 'UC-4': <MotifInfo with 5 motifs across 392 sites>,
 'Minimycin deletion': <MotifInfo with 7 motifs across 86 sites>,
 'RpoH': <MotifInfo with 3 motifs across 120 sites>,
 'Isorenieratene/streptamidine': <MotifInfo with 11 motifs across 89 sites>,

# Convert .eps files to .png

In [10]:
import os
import glob
from PIL import Image

def convert_eps_to_png_in_folder(folder_path):
    # Search for all .eps files in the folder and its subfolders
    for eps_file_path in glob.glob(os.path.join(folder_path, '**/*.eps'), recursive=True):
        # Create the .png file path by replacing the extension
        png_file_path = os.path.splitext(eps_file_path)[0] + '.png'
        
        # Convert the .eps file to .png
        img = Image.open(eps_file_path)
        img.save(png_file_path, 'PNG')

# Usage
convert_eps_to_png_in_folder('motifs/')

# Compare motifs to known databases using TOMTOM
For some reason it doesn't work to just read the function from motifs.py, so I copied it here.

We might want to adjust the thresholds and the databases used, according to other papers (See lab journal 17.01.23). The two papers using this method used slightly different parameters. I think the default is fine, as long as we state them in the methods section.

In [17]:
def compare_motifs2(
    motif_info: Optional[MotifInfo] = None,
    motif_file: Optional[str] = None,
    motif_db: Optional[str] = None,
    outdir: Optional[str] = None,
    force=False,
    verbose=True,
    evt=0.001,
):
    """
    Compare a MEME motif against external motif databases

    Parameters
    ----------
    motif_info: Optional[MotifInfo]
        MotifInfo object. Either 'motif_info' or 'motif_file' must be provided.
        'motif_info' takes precedence over 'motif_file'.
    motif_file: Optional[os.PathLike]
        Txt file generated by MEME from a motif search.
    motif_db: Optional[Union[List,str]]
        Name or path to MEME formatted databases
    outdir: Optional[os.PathLike]
        Output directory for TOMTOM comparisons
    force: bool
        Force execution of TOMTOM even if output already exists (default: False)
    verbose: bool
        Show steps in verbose output (default: True)
    evt: float
        E-value threshold (default: 0.001)

    Returns
    -------

    """

    if motif_info:
        motif_file = motif_info.file
    elif motif_file is None:
        raise ValueError("Either 'motif_info' or 'motif_file' must be provided")

    # Ensure motif file exists
    if not os.path.isfile(motif_file):
        raise FileNotFoundError(f"File does not exist: '{motif_file}'")

    # Get motif DB locations
    motif_db_dir = next(iter(pymodulon.data.meme.__path__))
    default_dbs = [
        "collectf",
        "dpinteract",
        "prodoric",
        "regtransbase",
        "SwissRegulon_e_coli",
    ]

    if motif_db is None:
        motif_db = default_dbs
        motif_db = [os.path.join(motif_db_dir, db + ".meme") for db in motif_db]
    elif isinstance(motif_db, str):
        motif_db = [motif_db]

    for db in motif_db:
        if db in default_dbs:
            motif_db = [os.path.join(motif_db_dir, db + ".meme") for db in motif_db]
        elif not os.path.isfile(db):
            raise FileNotFoundError(f"File does not exist: '{db}'")

    # Handle output directory
    if outdir is None:
        outdir = os.path.join(os.path.dirname(motif_file), "tomtom_consensus")

    # Run TOMTOM
    if not os.path.isdir(outdir) or force:
        cmd = [
            "tomtom",
            "-oc",
            outdir,
            "-thresh",
            str(evt),
            "-incomplete-scores",
            "-png",
            motif_file,
        ] + motif_db

        if verbose:
            print("Running command:", " ".join(cmd))

        res = subprocess.check_call(cmd)
        if res != 0:
            cmd_str = " ".join(cmd)
            raise RuntimeError(
                f"TOMTOM failed to execute the following command: {cmd_str}"
            )
    else result = _parse_tomtom(outdir)

    # Save result
    if motif_info:
        motif_info.matches = result
    return result

In [ ]:
import subprocess

for i in ica_data.motif_info:
    try:
        compare_motifs2(ica_data.motif_info[i], verbose=False)
    except subprocess.CalledProcessError:
        print(f"An error occurred with {i}. Skipping...")
        continue

In [13]:
ica_data.motif_info['Paulomycin-1'].sites

rel_position    pvalue                            site_seq  \
motif   operon                                                              
MEME-1  Op156           NaN       NaN                                 NaN   
        Op1616          NaN       NaN                                 NaN   
        Op1617          NaN       NaN                                 NaN   
        Op1618          NaN       NaN                                 NaN   
        Op2596          NaN       NaN                                 NaN   
...                     ...       ...                                 ...   
MEME-10 Op438           NaN       NaN                                 NaN   
        Op439           NaN       NaN                                 NaN   
        Op440           NaN       NaN                                 NaN   
        Op441           NaN       NaN                                 NaN   
        Op442            43  3.77e-10  CGTCGACGAGGTGGCTCCCGACCAGGAGAGCAGG   

                                  genes               locus_tags  start_pos  \
motif   operon                                                                
MEME-1  Op156                      xylB              XNR_RS00955     233806   
        Op1616  XNR_RS10865,XNR_RS10870  XNR_RS10865,XNR_RS10870    2562222   
        Op1617              XNR_RS31180              XNR_RS31180    2562455   
        Op1618              XNR_RS31185              XNR_RS31185    2563259   
        Op2596              XNR_RS17100              XNR_RS17100    3932478   
...                                 ...                      ...        ...   
MEME-10 Op438               XNR_RS02915              XNR_RS02915     696705   
        Op439                 rfbA,rfbB  XNR_RS02920,XNR_RS02925     698918   
        Op440          XNR_RS02930,dhbA  XNR_RS02930,XNR_RS02935     699149   
        Op441               XNR_RS02940              XNR_RS02940     701380   
        Op442   XNR_RS02945,XNR_RS02950  XNR_RS02945,XNR_RS02950     702159   

               strand  
motif   operon         
MEME-1  Op156       +  
        Op1616      -  
        Op1617      -  
        Op1618      -  
        Op2596      -  
...               ...  
MEME-10 Op438       -  
        Op439       -  
        Op440       +  
        Op441       +  
        Op442       +  

[210 rows x 7 columns]

# Generate a table summarizing results of motif search

In [15]:
keys = list(ica_data.motif_info.keys())

motif_table=pd.DataFrame(index=keys)

for i in motif_table.index:
    a = ica_data.motif_info[i].sites
    
    if i != 'PvdS':
        motif_table.loc[i, 'Oplists'] = a.loc['MEME-1'][-a.loc['MEME-1','pvalue'].isna()].index
    else:
        motif_table.loc[i, 'Oplists'] = a.loc['MEME-4'][-a.loc['MEME-4','pvalue'].isna()].index

for i in motif_table.index:
    motif_table.loc[i,'Op_num']=len(motif_table.loc[i,'Oplists'])
   
    motif_table.loc[i,'Op_total_num']=len(ica_data.view_imodulon(i).operon.unique())


    gene_num=0
    
    for j in motif_table.loc[i,'Oplists']:
        gene_num = gene_num+len(ica_data.view_imodulon(i).operon[ica_data.view_imodulon(i).operon == j])
        
    motif_table.loc[i,'motif_gene_num'] = gene_num
    
    motif_table.loc[i,'gene_total_num']=len(ica_data.view_imodulon(i))
    
    if i != 'PvdS':
        motif_table.loc[i,'consensus'] = ica_data.motif_info[i].motifs.loc['MEME-1','consensus']
        motif_table.loc[i,'width'] = ica_data.motif_info[i].motifs.loc['MEME-1','width']

    else:
        motif_table.loc[i,'consensus'] = ica_data.motif_info[i].motifs.loc['MEME-4','consensus']
        motif_table.loc[i,'width'] = ica_data.motif_info[i].motifs.loc['MEME-4','width']

In [16]:
motif_table.drop(columns=['Oplists'])

,Op_num,Op_total_num,motif_gene_num,gene_total_num,consensus,width
BGC-19-20 deletion,42.0,42.0,53.0,53.0,WBSYCGWCGDCSNCGGYGACSWSSWSSDSSTSSWSGVMSD,40
Glutamine,20.0,20.0,26.0,26.0,TSSYCRCCGHSSTCVCCGMSMKSSHCGVCSHSGSCSTS,38
Prophages,11.0,12.0,19.0,20.0,KCVYCCWCVRGHTCGACGGGGWSMTSSWCSRSYWCNTCG,39
Surugamide repressor,4.0,4.0,7.0,7.0,CASVSCTTCRVHCTWKYTGAYATWCAACTASTAGAAC,37
ArsR,2.0,2.0,4.0,4.0,TCTTGTTTCGACATGGGTCTATGTTGACAGCCTTCGATG,39
...,...,...,...,...,...,...
Quorum sensing,105.0,108.0,131.0,134.0,KCSASGHCSNSSDCGNCGWSSWSSWSGVCSAGSTCS,36
Serine protease,26.0,26.0,27.0,27.0,GCGGYSGSGGYGRSSGCGGBGGCGAGS,27
Surugamide,62.0,62.0,89.0,89.0,GGYGVYSRSSRCGRBCGCSGCSRCGRCCGYSR,32
Amino acid transport,107.0,107.0,119.0,119.0,HSSNSGHSVNGGMSGHSCDGSWSGBCSDSGMCSWSGHCSW,40


In [19]:
motif_table.to_csv("../../data/processed/modulome/motif_table.csv")         

In [20]:
# how many rows are in the motif table?
len(motif_table)

66

# Save the ica object

In [21]:
data_dir = path.join('../..','data','processed','modulome')
save_to_json(ica_data, path.join(data_dir, 'salb.json.gz'))

# convert .xml files for use with Difflogo

In [31]:
import pandas as pd
import os

def read_pwm(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Find the line that starts with "MOTIF" and ends with "MEME-1"
    motif_line = next((i for i, line in enumerate(lines) if line.startswith('MOTIF') and line.endswith('MEME-1\n')), None)

    if motif_line is None:
        print(f"No 'MOTIF' line ending with 'MEME-1\\n' found in file {file_path}. Skipping this file.")
        return None

    # Find the next line that starts with "letter-probability matrix:"
    start_line = next(i for i, line in enumerate(lines[motif_line:]) if line.startswith('letter-probability matrix:'))

    # Find the next line that starts with "MOTIF"
    end_line = next((i for i, line in enumerate(lines[motif_line + start_line:]) if line.startswith('MOTIF')), None)

    # If end_line is None, set it to the length of lines
    if end_line is None:
        end_line = len(lines) - motif_line - start_line

    # Read the lines between start_line and end_line into a DataFrame
    pwm = pd.DataFrame([line.split() for line in lines[motif_line + start_line + 1:motif_line + start_line + end_line]], dtype=float)

    # Transpose the DataFrame
    pwm = pwm.transpose()

    return pwm

# Get a list of all directories in './motifs/'
directories = [d for d in os.listdir('./motifs/') if os.path.isdir(os.path.join('./motifs/', d))]

# Process each directory
for directory in directories:
    # Use the function to read the PWM from the file
    pwm = read_pwm(f'./motifs/{directory}/output.txt')
    if pwm is None:
        continue
    # Remove columns with all NaN values
    pwm = pwm.dropna(axis=1, how='all')
    # Save the PWM to a file
    pwm.to_csv(f'../../data/interim/modulome/pmws/{directory}_pwm.txt', sep='\t', index=False, header=False)

No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/Sugar-2/output.txt. Skipping this file.
No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/Phospholipid/output.txt. Skipping this file.
No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/Valine/output.txt. Skipping this file.
No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/Potassium/output.txt. Skipping this file.
No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/Fructose/output.txt. Skipping this file.
No 'MOTIF' line ending with 'MEME-1\n' found in file ./motifs/hrt_Complex/output.txt. Skipping this file.
